In [1]:
import sys
sys.path.append('..//')
import merge_consensus as mc
import pod5
import pod5_importer as p5i
#import tracemalloc
import gc
import pandas as pd
import numpy as np
import ast
import pickle
import datetime
import os
#tracemalloc.start()
import psutil

In [2]:
pod5_str = "../../data/large_working_directory/merged_pod5/merged.pod5"

In [6]:
import subprocess
def invoke_pod5_importer(path):
    output = subprocess.run(["python", "pod5_importer.py", "{}".format(path)])
    print(output)
    return output.returncode

In [7]:
pod5_df = invoke_pod5_importer(pod5_str)

running...


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




CompletedProcess(args=['python', 'pod5_importer.py', '../../data/large_working_directory/merged_pod5/merged.pod5'], returncode=1)


1

In [5]:
!python pod5_importer.py -h

running...
usage: pod5_importer.py [-h] path

pod5 loading

positional arguments:
  path        Path to Pod5 file.

optional arguments:
  -h, --help  show this help message and exit


In [3]:
def memory_report():
    #snapshot = tracemalloc.take_snapshot()
    #top_stats = snapshot.statistics('lineno')
    proc = psutil.Process(os.getpid())
    rss = proc.memory_info().rss/(1*10**9)
    vms= proc.memory_info().vms/(1*10**9)
    shared = proc.memory_info().shared/(1*10**9)
    data = proc.memory_info().data/(1*10**9)
    #size,peak= tracemalloc.get_traced_memory()
    print("[ Memory Report {} ]------------------------------------------------".format(datetime.datetime.now()))
    print("RSS: {} gb, VMS: {} gb, SHARED: {} gb, DATA: {} gb".format(rss, vms, shared, data))
    #print("Size: {} gb, Peak {} gb".format(size/(1*10**9),peak/(1*10**9)))
    #print("[ Top 5 Memory Uses ]")
    #for stat in top_stats[:5]:
    #    print(stat)
    print("--------------------------------------------------------------------------------------------")

In [4]:
memory_report()

[ Memory Report 2024-05-13 17:38:04.667684 ]------------------------------------------------
RSS: 0.190906368 gb, VMS: 3.750793216 gb, SHARED: 0.07929856 gb, DATA: 1.68767488 gb
--------------------------------------------------------------------------------------------


In [7]:
df = pd.DataFrame(mc.load_pod5_data(pod5_str))

<class 'numpy.ndarray'>


In [1]:
def testt():
    memory_report()
    df2 = pd.read_pickle('export_merge_pod5.pickle')
    print(sys.getsizeof(df2))
    memory_report()
    
    return None

In [5]:
testt()
memory_report()

[ Memory Report 2024-05-13 17:38:07.978003 ]------------------------------------------------
RSS: 0.190906368 gb, VMS: 3.750793216 gb, SHARED: 0.07929856 gb, DATA: 1.68767488 gb
--------------------------------------------------------------------------------------------
89178727
[ Memory Report 2024-05-13 17:38:12.617379 ]------------------------------------------------
RSS: 8.398368768 gb, VMS: 11.957264384 gb, SHARED: 0.080216064 gb, DATA: 9.894146048 gb
--------------------------------------------------------------------------------------------
[ Memory Report 2024-05-13 17:38:12.876167 ]------------------------------------------------
RSS: 4.825284608 gb, VMS: 8.383905792 gb, SHARED: 0.080347136 gb, DATA: 6.320787456 gb
--------------------------------------------------------------------------------------------


In [6]:
size,peak= tracemalloc.get_traced_memory()

(498283, 8168251563)

In [5]:
def tryscale(scale):
    print(scale)
    data_list = []
    for i in range(1):
        data_list = []
        with pod5.Reader(pod5_str) as pod5_file:
            memory_report()
            reads=list(pod5_file.reads())

            for i in range(len(reads[:scale])):
                read = reads[i]
                # save the sequence id, signal, and frequency to a dict
                data_dict = getreaddict(read)
                # append the dict to the list
                data_list.append(data_dict)
            pod5_file.close()
            memory_report()
        memory_report()

In [ ]:
def getreaddict(read):        
    return data_dict
    seq_id = read.read_id
    signal = read.signal_pa
    freq = read.run_info.sample_rate
    data_dict = {'seq_id': str(seq_id),
                 'signal': signal,
                 'freq': freq}

In [3]:
!pip install "dask[dataframe]"

In [9]:
memory_report()

[ Top 5 Memory Uses ]--------------------------
/home/sebastian/.conda/envs/xna_seq/lib/python3.8/tracemalloc.py:65: size=18.7 KiB, count=299, average=64 B
/home/sebastian/.conda/envs/xna_seq/lib/python3.8/tracemalloc.py:532: size=15.5 KiB, count=325, average=49 B
/home/sebastian/.conda/envs/xna_seq/lib/python3.8/site-packages/pod5/reader.py:106: size=13.3 KiB, count=81, average=168 B
/home/sebastian/.conda/envs/xna_seq/lib/python3.8/site-packages/IPython/core/builtin_trap.py:70: size=9248 B, count=1, average=9248 B
/home/sebastian/.conda/envs/xna_seq/lib/python3.8/abc.py:102: size=9217 B, count=95, average=97 B


In [19]:
read_list_dict = []
for read in reads_list:
    seq_id = read.read_id
    signal = read.signal_pa
    freq = read.run_info.sample_rate
    data_dict = {'seq_id': str(seq_id),
                 'signal': signal,
                 'freq': freq}
    read_list_dict.append(data_dict)
memory_report()

RuntimeError: ArrowTableHandle has been closed!